In [1]:
##%matplotlib inline
import csv
import nltk
#from nltk.stem.porter import PorterStemmer
from stem import IndonesianStemmer
import string
#import os
import re
from collections import defaultdict
import multiprocessing
import dill
import utils

In [4]:
#stopwords = nltk.corpus.stopwords.words('indonesian')

In [3]:
%%time
token_dict = defaultdict(lambda: None)
product_category = defaultdict(lambda: None)

with open('/data/python-scrapy-data-mining/corpus.csv') as raw_file:
    for (prod_id, title, url) in csv.reader(raw_file):

        url = url.split('/')
        if url[3] != 'p':
            continue
        cat = '/'.join(url[4:-1])
        product_category[int(prod_id)] = cat
        token_dict[int(prod_id)] = utils.tokenize_and_stem_ID(
            title.lower().translate(None, string.punctuation))

CPU times: user 12.9 s, sys: 100 ms, total: 13 s
Wall time: 12.9 s


stem.py:45: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return letter in [u'a', u'e', u'i', u'o', u'u']


In [4]:
%time collection = nltk.text.TextCollection(token_dict.values())

CPU times: user 728 ms, sys: 24 ms, total: 752 ms
Wall time: 742 ms


In [5]:
#get a list of unique terms
print "Toal products:", len(token_dict)
unique_terms = list(set(collection))
print "Unique terms found: ", len(unique_terms)

Toal products: 228562
Unique terms found:  101641


In [8]:
# Function to create a TF*IDF vector for one document.  For each of
# our unique words, we have a feature which is the td*idf for that word
# in the current document
def TFIDF(document):
    global collection
    document_tfidf = {}
    for word in unique_terms:
        if word not in document:
            continue
        ifidf = collection.tf_idf(word, document)
        if ifidf > 0.01:
            document_tfidf[word] = ifidf
    return document_tfidf

In [7]:
TFIDF(token_dict[1])

{'akses': 0.46101537318034874,
 'bahan': 0.23076973574225682,
 'case': 0.1614701117579361,
 'dan': 0.17965810369248522,
 'design': 0.2972940969474257,
 'eleg': 0.3084819637446488,
 'flip': 0.23045209262253283,
 'galaxy': 0.18783789178329843,
 'kualitas': 0.26221271931655943,
 'kulit': 0.21396189366829269,
 'leather': 0.21236075889631367,
 'mudah': 0.33223294845052076,
 'pasang': 0.2986928087050054,
 'port': 0.2644973481702362,
 'pu': 0.3362691074861176,
 'ringan': 0.3671310413755716,
 's5': 0.2988350765690476,
 'saat': 0.36170618875590954,
 'samsung': 0.16557050769599174,
 'sintesis': 0.5109522953925355,
 'tinggi': 0.2970194462903365,
 'tombol': 0.33963765167492316}

In [22]:
%%time
def term_idf(terms):
    global collection
    return [(t, collection.idf(t)) for t in terms]

pool = multiprocessing.Pool(processes=8)
terms = pool.map(term_idf, utils.chunks(unique_terms, 8))

CPU times: user 2.89 s, sys: 1.99 s, total: 4.88 s
Wall time: 25min 49s


In [30]:
import operator
#%time terms = reduce(operator.add, terms, [])
terms = sorted(terms, key=lambda t: -t[1])
print "Total unique terms:", len(terms)

Total unique terms: 101633


In [34]:
# Inverted Document Frequencies (of the unique terms):
terms[:10] + ["..."] + terms[-10:]

[('fawn', 12.339562787303892),
 ('tsukino', 12.339562787303892),
 ('5quot', 12.339562787303892),
 ('400410', 12.339562787303892),
 ('daiich', 12.339562787303892),
 ('asustrixgtx750tioc2gd5', 12.339562787303892),
 ('lodyne', 12.339562787303892),
 ('inlite', 12.339562787303892),
 ('n7100note', 12.339562787303892),
 ('ds1015', 12.339562787303892),
 '...',
 ('sepatu', 3.674984608974596),
 ('usb', 3.6517832953121205),
 ('samsung', 3.642551169311818),
 ('black', 3.6282841721734576),
 ('1', 3.6197553358259373),
 ('case', 3.552342458674594),
 ('jam', 3.5344875434332073),
 ('for', 3.494073550550618),
 ('tas', 3.3190520876119853),
 ('original', 3.1732788015246287)]

In [44]:
%time backup_object(terms, "terms.pkl")

CPU times: user 1.99 s, sys: 328 ms, total: 2.32 s
Wall time: 4.93 s


In [73]:
#%time len([TFIDF(token_dict[k]) for k in token_dict.keys()[1000:2000]])

CPU times: user 2min 9s, sys: 844 ms, total: 2min 10s
Wall time: 2min 9s


1000

In [12]:
%%time
# Vectorize products (into a list of tuples (product_id, vector representation)):
### vectors = [(prod_id,TFIDF(token_dict[prod_id])) for prod_id in token_dict.keys()]

def product_vectors(product_ids):
    global token_dict
    return [(prod_id,TFIDF(token_dict[prod_id])) for prod_id in product_ids]

pool = multiprocessing.Pool(processes=4)
vectors = pool.map(product_vectors, utils.chunks(token_dict.keys(), 4))

CPU times: user 8.34 s, sys: 2.31 s, total: 10.7 s
Wall time: 1h 12min 55s


/home/nad2000/venv/lib/python2.7/site-packages/ipykernel/__main__.py:8: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/home/nad2000/venv/local/lib/python2.7/site-packages/nltk/text.py:566: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  matches = len([True for text in self._texts if term in text])
/home/nad2000/venv/lib/python2.7/site-packages/ipykernel/__main__.py:8: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/home/nad2000/venv/lib/python2.7/site-packages/ipykernel/__main__.py:8: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/home/nad2000/venv/local/lib/python2.7/site-packages/nltk/text.py:566: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - int

In [27]:
vectors = sum(vectors, [])

In [123]:
#dill.dump(vectors, open("vectors.pkl", "wb"))

In [29]:
%%time
# Add norm values:
vectors[:] = [(prod_id, (v, utils.norm(v))) for prod_id, v in vectors]
products = dict(vectors)

CPU times: user 2.95 s, sys: 292 ms, total: 3.24 s
Wall time: 5.53 s


In [33]:
%%time
dill.dump(products, open("products.pkl", "wb"))

CPU times: user 17.1 s, sys: 0 ns, total: 17.1 s
Wall time: 17 s
